In [2]:
import nltk
import json
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from gensim import corpora, models, similarities

C:\Users\yazquez\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Carga de los datos

Funcion para cargar las sinopsis de las películas y los metadados de las mismas (género, pais, etc). Nos quedaremos con las películas para las cuales tengamos tanto los metadatos como la sinpsis. La lista de películas será un objeto de tipo **list** y los datos de cada película un diccionario.

In [20]:
def load_movies(max_movies=90000):
    def load_plot_summaries():
        # plot_summaries_file = open("data/plot_summaries.fake.txt", "r")
        plot_summaries_file = open("data/plot_summaries.txt", "r", encoding="utf8")
        plot_summaries = dict()
        for plot_summary_line in plot_summaries_file:
            plot_summary_data = plot_summary_line.split('\t')
            # Summaries structure
            # [0] Wikipedia movie ID
            # [1] Summary plot
            plot_summary = dict()
            plot_summary['id'] = plot_summary_data[0]
            plot_summary['summary'] = plot_summary_data[1]

            plot_summaries[plot_summary['id']] = plot_summary

        return plot_summaries

    print("Cargando datos de peliculas...")

    plot_summaries = load_plot_summaries()

    # metadata_file = open("data/movie.metadata.fake.tsv", encoding="utf8")
    metadata_file = open("data/movie.metadata.tsv", "r", encoding="utf8")
    movies = []

    movies_count = 0

    for metadata_line in metadata_file:
        movie_metadata = metadata_line.split('\t')

        # Metadata structure
        # [0] Wikipedia movie ID
        # [1] Freebase movie ID
        # [2] Movie name
        # [3] Movie release date
        # [4] Movie box office revenue
        # [5] Movie runtime
        # [6] Movie languages (Freebase ID:name tuples)
        # [7] Movie countries (Freebase ID:name tuples)
        # [8] Movie genres (Freebase ID:name tuples)

        id = movie_metadata[0]

        # Añadimos la pelicula solo si tiene sinopsis, incluimos una lista con las claves de los generos
        if (id in plot_summaries) & (movies_count < max_movies):
            movies_count += 1
            movie = dict()
            movie['id'] = id
            movie['name'] = movie_metadata[2]
            movie['date'] = movie_metadata[3]
            movie['genres'] = list(json.loads(movie_metadata[8].replace("\"\"", "\"").replace("\"{", "{").replace("}\"", "}")).keys())
            movie['summary'] = plot_summaries[id].get('summary')
            movies.append(movie)

    print("Número de películas cargadas:", len(movies))
    
    return movies

Usando la funcíon anterior cargamos los datos, dado que el proceso es bastante pesado solo cargaremos un subconjunto de las mismas

In [21]:
movies = load_movies(100)

Cargando datos de peliculas...
Número de películas cargadas: 100


# Procesado de las sinópsis

A continuación procesamos las sinopsis, quedándonos con las palabras diferentes que encontramos en cada una de ellas, adicionalmente las vamos a transformar en sus raices (stemmer) y obviaremos las **stopwords** y los nomrbres propios, dado que no aportan significado.

Primero definimos una función para hacerlo

In [22]:
def get_global_texts(movies):
    def get_words(text):
        def add_word(word):
            word = word.lower()
            if word not in stop_words:
                words.append(stemmer.stem(word))


        words = []
        for chunk in nltk.ne_chunk(nltk.pos_tag(tokenizer.tokenize(text))):
            # nltk.word_tokenize    devuelve la lista de palabras que forman la frase (tokenización)
            # nltk.pos_tag          devuelve el part of speech (categoría) correspondiente a la palabra introducida
            # nltk.ne_chunk         devuelve la etiqueta correspondiente al part of speech (POC)
            try:
                if chunk.label() == 'PERSON':
                    # PERSON es un POC asociado a los nombres propios, los cuales no vamos a añadir
                    pass
                else:
                    for c in chunk.leaves():
                        add_word(c[0])
            except AttributeError:
                add_word(chunk[0])

        return words

    print("Extrayendo palabras de los textos...")
    tokenizer = RegexpTokenizer(r'\w+')
    stop_words = set(stopwords.words('english'))
    stemmer = SnowballStemmer("english")

    global_texts = []

    [global_texts.append(get_words(movie['summary'])) for movie in movies]

    return global_texts

Y la ejecutamos, tendremos una lista de listas, en la que para cada película tendremos las palabras que definen su sinopsis

In [23]:
global_texts = get_global_texts(movies)

Extrayendo palabras de los textos...


# Creación del diccionaro

El diccionario está formado por la concatenación de todas las palabras que aparecen en alguna sinopsis (modo texto) de alguna de las peliculas. Básicamente esta función mapea cada palabra única con su identificador. Es decir, si tenemos N palabras, lo que conseguiremos al final es que cada película sea representada mediante un vector en un  espacio de N dimensiones.

Para ello, partiendo de la lista creada en el paso anterior, usaremos la función **corpora** del paquete **gensim**. 

In [27]:
dictionary = corpora.Dictionary(global_texts)
dictionary

In [26]:
dictionary

# Creación del Corups

Crearemos un corpus con la colección de todos los resúmenes previamente pre-procesados y transformados usando el diccionario.

In [31]:
corpus = [dictionary.doc2bow(text) for text in global_texts]

A modo de ejemplo, mostramos las 5 primeras entradas de la primera pelicula

In [32]:
corpus[0][:5]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]

In [ ]:
# Creación del TFID

In [ ]:
def create_tfidf(corpus):
    print("Creación del Modelo Espacio-Vector Tf-Idf")
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    return corpus_tfidf

corpus_tfidf = create_tfidf(corpus)

# Creación del Modelo LSI

Para ello vamos a definir una función auxiliar y posteriormente la invocaremos

In [ ]:
def create_lsi_model(corpus_tfidf, dictionary, total_lsa_topics):
    print("Creación del modelo LSA: Latent Semantic Analysis")
    lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=total_lsa_topics)
    similarity_matrix = similarities.MatrixSimilarity(lsi[corpus_tfidf])
    return lsi, similarity_matrix

(lsi, similarity_matrix) = create_lsi_model(corpus_tfidf, dictionary, TOTAL_LSA_TOPICS)